In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
import json
import re
from collections import defaultdict
import processing_pdf

Define the input and output path.

In [3]:
# path to the folder containing PDF files (1512, removed researchpaper1.pdf)
dataset_path = "F:\Datasets/"
# output folder and file name
# output_path = "dataset"
# processing_pdf.clear_processed_folder(output_path)

Step 0: Remove duplicated files, only keep the papers (941) with newest version

In [7]:
# Function to extract the base filename without the version suffix
def get_base_filename(filename):
    match = re.match(r'^(.*?)v\d\.pdf+$', filename)
    if match:
        return match.group(1)
    else:
        return filename

# Function to find the newest version of each file
def find_newest_versions(folder):
    files_by_base_name = defaultdict(list)

    # Group files by base filename
    for filename in os.listdir(folder):
        base_name = get_base_filename(filename)
        files_by_base_name[base_name].append(filename)

    # Find the newest version of each file
    newest_versions = []
    for base_name, filenames in files_by_base_name.items():
        newest_version = max(filenames, key=lambda x: int(re.search(r'v(\d+)\.pdf$', x).group(1)))
        newest_versions.append(newest_version)

    return newest_versions

# Function to remove duplicated files
def remove_duplicates(folder):
    newest_versions = find_newest_versions(folder)

    for filename in os.listdir(folder):
        if filename not in newest_versions:
            file_path = os.path.join(folder, filename)
            if os.path.isfile(file_path):
                os.remove(file_path)

remove_duplicates(dataset_path)

Step 1: Breakdown PDF content by sections and subsections.

In [4]:
def process_one_pdf(file_path):
    doc, total_text, _ = processing_pdf.open_file(file_path)
    table_of_content = doc.get_toc()

    if len(table_of_content) > 0:
        print("Auto generated table of content:")
        display(table_of_content)
        # separate content into sections
        _, json_dict = processing_pdf.separate_content(total_text, table_of_content)
        return json_dict
    # some papers have not table of content
    #if len(table_of_content) == 0:
    #    print("The paper has not table of content. Need to use regular expression to map table of content.")
    #    table_of_content = processing_pdf.auto_find_toc(doc)
    #    display(table_of_content)
    
    return {}

In [5]:
# number of PDFs used for training data preparation
dataset_limit = 20

file_count = 0
all_json_dicts = []
# Loop through files in the folder
for file_name in os.listdir(dataset_path):
    file_path = os.path.join(dataset_path, file_name)
    # Check if it's a file and if it has a ".pdf" extension
    if os.path.isfile(file_path) and file_name.endswith('.pdf'):
        # Process the PDF file
        print("Processing PDF file:", file_name)
        json_dict = process_one_pdf(file_path)
        if len(json_dict) > 0:
            all_json_dicts.extend(list(json_dict.values()))
            file_count += 1
            print("Done with PDF file:", file_name)
            print("# of sections:", len(json_dict.values()))
            print("Total # of sections:", len(all_json_dicts), "Total # of files:", file_count)
            print(60*"=")
        
        # Terminate when reaching dataset limit
        if file_count >= dataset_limit:
            break

# get_summaries(all_json_dicts)
json_list = json.dumps(all_json_dicts)
full_name = "dataset/dataset_eval.json"
with open(full_name, "w") as jsonfile: 
    jsonfile.write(json_list)

Processing PDF file: 1901.00100v1.pdf
Processing PDF file: 1901.00101v1.pdf
Auto generated table of content:


[[1, 'I Introduction', 1],
 [1, 'II Related Work', 2],
 [1, 'III Safety-Guided RRT  via Probabilistically Safe Corridors', 2],
 [2, 'III-A Gaussian Mixture Modeling of Configuration Spaces', 2],
 [3, 'III-A.1 Learning Gaussian Mixtures', 3],
 [3, 'III-A.2 Confidence Regions of Gaussian Mixtures', 3],
 [2, 'III-B Probabilistically Safe Corridors', 4],
 [2, 'III-C Guided Steering via Safe Corridors', 5],
 [3, 'III-C.1 Tree Extension in the Configuration Space', 5],
 [3, 'III-C.2 Tree Extension in the Task Space', 6],
 [3, 'III-C.3 GMM-based Biased Sampling', 6],
 [1, 'IV Results', 6],
 [2, 'IV-A Learning Gaussian Mixture Models', 6],
 [2, 'IV-B 2DoF Planar Manipulator', 6],
 [2, 'IV-C 7DoF Manipulator in 3D Space', 8],
 [2, 'IV-D Physical Robot Experiments', 8],
 [1, 'V Discussion', 9],
 [1, 'References', 9]]

starting looking for all the sections according to the provided section title info...
Done with PDF file: 1901.00101v1.pdf
# of sections: 7
Total # of sections: 7 Total # of files: 1
Processing PDF file: 1901.00102v1.pdf
Auto generated table of content:


[[1, '1 Introduction', 2],
 [1, '2 The bremsstrahlung cross section with screening potential', 4],
 [1, '3 Comparing with the Bethe-Heitler formula', 14],
 [1, '4 Summary', 21]]

starting looking for all the sections according to the provided section title info...
Done with PDF file: 1901.00102v1.pdf
# of sections: 5
Total # of sections: 12 Total # of files: 2
Processing PDF file: 1901.00103v1.pdf
Processing PDF file: 1901.00104v1.pdf
Auto generated table of content:


...

starting looking for all the sections according to the provided section title info...
Done with PDF file: 1901.00130v1.pdf
# of sections: 6
Total # of sections: 117 Total # of files: 18
Processing PDF file: 1901.00131v4.pdf
Auto generated table of content:


[[1, '1 Introduction', 1],
 [1, '2 Martingale approximations', 4],
 [1, '3 Main abstract theorem', 8],
 [2, '3.1 Verifying condition (b) in Theorem ??', 10],
 [2, '3.2 Verifying condition (a) in Theorem ??', 11],
 [1, '4 Application to Lorentz gases', 12],
 [2, '4.1 Setting and main result for Lorentz gases', 12],
 [2, '4.2 Proof of Theorem ??', 13]]

starting looking for all the sections according to the provided section title info...
Done with PDF file: 1901.00131v4.pdf
# of sections: 5
Total # of sections: 122 Total # of files: 19
Processing PDF file: 1901.00132v1.pdf
Auto generated table of content:


[[1, 'I Introduction and related work', 1],
 [1, 'II A real-world dataset', 1],
 [1, 'III Forecasting technique', 2],
 [1, 'IV Numerical results', 2],
 [2, 'IV-A Performance metrics', 2],
 [2, 'IV-B Results', 3],
 [1, 'V Conclusion and Future Work', 4],
 [1, 'References', 4]]

starting looking for all the sections according to the provided section title info...
Done with PDF file: 1901.00132v1.pdf
# of sections: 7
Total # of sections: 129 Total # of files: 20


Step 2: Generate GPT summary for each section and subsection as ground truth.

In [6]:
from gpt_summary import get_summaries

#file = f"dataset/test.json"
file = f"dataset/dataset_eval.json"
with open(file, encoding='utf-8') as f:
    data = json.load(f)
get_summaries(data)

with open(f"dataset/dataset_eval_ground_truth.json", 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)